In [2]:
import pandas as pd
import json
import re

In [33]:
df = pd.read_json('entities.json', lines=True)
df.head(10)

,name,url,entity_type,type,traded_as,industry,founded,founded_by,headquarters,area_served,...,employer,notable_work,died,cause_of_death,organization,awards,home_town,relatives,•_demonym,area_code(s)
0,Facebook,https://en.wikipedia.org/wiki/Facebook,company,Public,"[NASDAQ:_FB (Class A), NASDAQ-100 component, S...","[Social media, Advertising]","February_4, 2004; 15 years ago_(2004-02-04) in...","[Mark Zuckerberg, Eduardo Saverin, Andrew McCo...","1 Hacker Way (aka 1601 Willow Road), Menlo Par...","United States (2004–present)Worldwide, except ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Summify,https://en.wikipedia.org/wiki/Summify,company,NaN,NaN,"[news aggregation, social media]",NaN,"[Mircea Paşoi, Cristian Strat]","Vancouver, British Columbia",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Periscope (app),https://en.wikipedia.org/wiki/Periscope_(app),company,"[Video streaming, Social networking service]",NaN,NaN,NaN,Kayvon Beykpour Joe Bernstein Aaron Wasserma...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Vine (service),https://en.wikipedia.org/wiki/Vine_(service),company,Video sharing,NaN,NaN,NaN,"[Dom Hofmann, Rus Yusupov, Colin Kroll]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pinterest,https://en.wikipedia.org/wiki/Pinterest,company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Snapchat,https://en.wikipedia.org/wiki/Snapchat,company,"[Photo sharing, Instant messaging, Video chat,...",NaN,NaN,NaN,"[Evan Spiegel, Bobby Murphy, Reggie Brown]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,WeChat,https://en.wikipedia.org/wiki/WeChat,company,"[Instant messaging, client]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Reddit,https://en.wikipedia.org/wiki/Reddit,company,NaN,NaN,"[Internet, media]","June_23, 2005; 14 years ago_(2005-06-23)[2]","[Steve Huffman, Aaron Swartz, Alexis Ohanian]","[San Francisco, California, U.S.]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Instagram,https://en.wikipedia.org/wiki/Instagram,company,NaN,NaN,NaN,NaN,"[Kevin Systrom, Mike Krieger]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Twitter,https://en.wikipedia.org/wiki/Twitter,company,Public,"[NYSE:_TWTR, S&P 500 Component]",Internet,NaN,"[Jack Dorsey, Noah Glass, Biz Stone, Evan Will...","San Francisco, California, United States",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Separate Entities JSON into separate tables

In [34]:
def dedup_df(df):
    return df.drop_duplicates('name').dropna(how='all',axis=1)

def fix_name(df):
    df['name'] = [z.split('(')[0].strip() for z in df['name']]
    return df
companies_df = fix_name(dedup_df(df[df['entity_type'] == "company"]))
persons_df = fix_name(dedup_df(df[df['entity_type']=='person']))
software_df = fix_name(dedup_df(df[df['entity_type'] == "programming_language"]))

persons_df = persons_df.drop([19])
companies_df = companies_df.drop([47, 66])
software_df = software_df.drop([12, 10, 30, 42, 51, 55, 57, 65])

In [35]:
print(companies_df.columns)
companies_df = companies_df[['name', 'url', 'entity_type', 'type', 'industry',
       'founded', 'founded_by', 'headquarters', 'key_people', 'products', 'revenue', 'owner', 'subsidiaries', 'website',
       'developer(s)', 'written_in','type_of_site', 'employees','coordinates', 'users', 'parent']]
companies_df

Index(['name', 'url', 'entity_type', 'type', 'traded_as', 'industry',
       'founded', 'founded_by', 'headquarters', 'area_served', 'key_people',
       'products', 'revenue', 'owner', 'subsidiaries', 'website',
       'developer(s)', 'stable_release', 'android', 'ios', 'written_in',
       'operating_system', 'available_in', 'license', 'alexa_rank',
       'android_(beta)', 'size', 'macos', 'windows_phone_8.1', 'symbian_v3',
       'symbian_v5', 'type_of_site', 'employees', 'advertising',
       'registration', 'windows_10', 'android_(alpha)', 'coordinates', 'users',
       'brands', 'parent', 'isin', 'operating_income', 'net_income',
       'total_assets', 'total_equity', 'number_of_employees'],
      dtype='object')


,name,url,entity_type,type,industry,founded,founded_by,headquarters,key_people,products,...,owner,subsidiaries,website,developer(s),written_in,type_of_site,employees,coordinates,users,parent
0,Facebook,https://en.wikipedia.org/wiki/Facebook,company,Public,"[Social media, Advertising]","February_4, 2004; 15 years ago_(2004-02-04) in...","[Mark Zuckerberg, Eduardo Saverin, Andrew McCo...","1 Hacker Way (aka 1601 Willow Road), Menlo Par...","[Mark Zuckerberg (Chairman and CEO), Sheryl Sa...","[Messenger, Watch, Portal]",...,"Facebook, Inc._","[Instagram, WhatsApp, Oculus, Calibra]",about.fb.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Summify,https://en.wikipedia.org/wiki/Summify,company,NaN,"[news aggregation, social media]",NaN,"[Mircea Paşoi, Cristian Strat]","Vancouver, British Columbia",NaN,NaN,...,NaN,NaN,summify.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Periscope,https://en.wikipedia.org/wiki/Periscope_(app),company,"[Video streaming, Social networking service]",NaN,NaN,Kayvon Beykpour Joe Bernstein Aaron Wasserma...,NaN,NaN,NaN,...,NaN,NaN,www.periscope.tv,Twitter,"[Go, server-side, [4]]",NaN,NaN,NaN,NaN,NaN
3,Vine,https://en.wikipedia.org/wiki/Vine_(service),company,Video sharing,NaN,NaN,"[Dom Hofmann, Rus Yusupov, Colin Kroll]",NaN,NaN,NaN,...,NaN,NaN,vine.co,"Vine Labs, Inc. (Twitter)",NaN,NaN,NaN,NaN,NaN,NaN
4,Pinterest,https://en.wikipedia.org/wiki/Pinterest,company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,www.pinterest.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Snapchat,https://en.wikipedia.org/wiki/Snapchat,company,"[Photo sharing, Instant messaging, Video chat,...",NaN,NaN,"[Evan Spiegel, Bobby Murphy, Reggie Brown]",NaN,NaN,NaN,...,NaN,NaN,www.snapchat.com,Snap Inc.,NaN,NaN,NaN,NaN,NaN,NaN
6,WeChat,https://en.wikipedia.org/wiki/WeChat,company,"[Instant messaging, client]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"[www.wechat.com, weixin.qq.com]",Tencent Holdings Limited,NaN,NaN,NaN,NaN,NaN,NaN
7,Reddit,https://en.wikipedia.org/wiki/Reddit,company,NaN,"[Internet, media]","June_23, 2005; 14 years ago_(2005-06-23)[2]","[Steve Huffman, Aaron Swartz, Alexis Ohanian]","[San Francisco, California, U.S.]","[Steve Huffman (co-founder and CEO), Jen Wong ...",NaN,...,"[Advance Publications, [1]]",NaN,www.reddit.com,NaN,"[Python, React[notes 3]]","[Social news, media aggregation]",400 (September 2018)[3],NaN,NaN,NaN
8,Instagram,https://en.wikipedia.org/wiki/Instagram,company,NaN,NaN,NaN,"[Kevin Systrom, Mike Krieger]",NaN,NaN,NaN,...,NaN,NaN,Instagram.com,"Facebook, Inc.",NaN,NaN,NaN,NaN,NaN,NaN
9,Twitter,https://en.wikipedia.org/wiki/Twitter,company,Public,Internet,NaN,"[Jack Dorsey, Noah Glass, Biz Stone, Evan Will...","San Francisco, California, United States","[Omid Kordestani, (Executive Chairman), Jack D...",NaN,...,NaN,"[Vine, Periscope, MoPub]","[twitter.com, ]",NaN,"[Java, Ruby, Scala, JavaScript]","[News, social networking service]","4,600 (September 2019)[1]",[37°46′36″N 122°25′00″W﻿ / ﻿37.7768°N 122.4166...,"[321 million active users, (February 2019)]",NaN


In [36]:
print(persons_df.columns.values)
persons_df

['name' 'url' 'entity_type' 'website' 'residence' 'alma_mater'
 'occupation' 'born' 'education' 'salary' 'net_worth' 'political_party'
 'board_member_of' 'spouse(s)' 'children' 'known_for' 'title'
 'nationality' 'citizenship' 'other_names' 'employer' 'notable_work'
 'died' 'cause_of_death' 'organization' 'awards' 'home_town' 'relatives']


,name,url,entity_type,website,residence,alma_mater,occupation,born,education,salary,...,citizenship,other_names,employer,notable_work,died,cause_of_death,organization,awards,home_town,relatives
16,Mike Schroepfer,https://en.wikipedia.org/wiki/Mike_Schroepfer,person,NaN,San Francisco Bay area,Stanford University,"[CTO, Facebook]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Sheryl Sandberg,https://en.wikipedia.org/wiki/Sheryl_Sandberg,person,NaN,"[Menlo Park, California, [1]]",NaN,"[COO, Facebook]","Sheryl Kara Sandberg (1969-08-28) August 28, 1...","[Harvard University, BA, MBA]","[US$, [2]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Chris Hughes,https://en.wikipedia.org/wiki/Chris_Hughes,person,NaN,NaN,Harvard University,Entrepreneur,"Christopher Hughes (1983-11-26) November 26, 1...",Phillips Academy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Evan Spiegel,https://en.wikipedia.org/wiki/Evan_Spiegel,person,NaN,"[Brentwood, Los Angeles, California, [1]]",Stanford University,Entrepreneur,"[Los Angeles, California]",Crossroads School for Arts and Sciences,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,Evan Williams,https://en.wikipedia.org/wiki/Evan_Williams_(I...,person,NaN,"San Francisco, California, US",NaN,Internet Entrepreneur,"[Clarks, Nebraska]",NaN,NaN,...,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,Omid Kordestani,https://en.wikipedia.org/wiki/Omid_Kordestani,person,NaN,"[Atherton, California, [3]]","[San Jose State University, BS, Stanford Unive...","[Executive Chairman, Twitter]","[Tehran, [1], Iran, [2]]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,Biz Stone,https://en.wikipedia.org/wiki/Biz_Stone,person,NaN,"Marin County, California",NaN,"[Creative director, software engineer, busines...",NaN,NaN,NaN,...,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,Jack Dorsey,https://en.wikipedia.org/wiki/Jack_Dorsey,person,NaN,"[San Francisco, California]","[University of Missouri–Rolla, New York Univer...","[Programmer, internet entrepreneur]","[[1], St. Louis, Missouri]",NaN,"[Square, Inc., Twitter, [2]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,Steve Huffman,https://en.wikipedia.org/wiki/Steve_Huffman,person,NaN,"San Francisco, California",University of Virginia,"[Co-founder and CEO of Reddit, Co-founder of H...",NaN,Computer Science,NaN,...,NaN,spez[1],Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,Alexis Ohanian,https://en.wikipedia.org/wiki/Alexis_Ohanian,person,Alexisohanian.com,"[West Palm Beach, Florida, citation needed]",University of Virginia,"[Entrepreneur, venture capitalist, investor]","Alexis Kerry Ohanian (1983-04-24) April 24, 19...",NaN,NaN,...,NaN,NaN,NaN,"[Reddit, Initialized Capital]",NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
software_df

,name,url,entity_type,type,website,stable_release,written_in,license,white_paper,code_repository,...,designed_by,developer,first_appeared,typing_discipline,filename_extensions,platform,implementation_language,os,initial_release,repository
26,JavaScript,https://en.wikipedia.org/wiki/JavaScript,programming_language,NaN,Mozilla,ECMAScript 2019[3]\n / June_2019; 5_month...,NaN,NaN,NaN,NaN,...,Brendan Eich,"[Netscape Communications Corporation, Mozilla ...","December_4, 1995; 23 years ago_(1995-12-04)[2]","[Dynamic, duck]","[.js, .mjs[4]]",NaN,NaN,NaN,NaN,NaN
27,Scala,https://en.wikipedia.org/wiki/Scala_(programmi...,programming_language,NaN,www.scala-lang.org,2.13.1\n / 11_June 2019; 5 months ago_(20...,NaN,"[Apache License 2.0, [2]]",NaN,NaN,...,Martin Odersky,Programming Methods Laboratory of École Polyte...,NaN,"[Inferred, static, strong, structural]",NaN,"[JVM, JavaScript (Scala.js), LLVM (Scala Nativ...",NaN,NaN,NaN,NaN
28,Ruby,https://en.wikipedia.org/wiki/Ruby_(programmin...,programming_language,NaN,www.ruby-lang.org,"[[1], [±]]",NaN,"[Ruby License, GPLv2, 2-clause BSD, [2], [3], ...",NaN,NaN,...,Yukihiro Matsumoto,NaN,NaN,"[Duck, dynamic, strong]",NaN,NaN,C,Cross-platform,NaN,NaN
29,Java,https://en.wikipedia.org/wiki/Java_(programmin...,programming_language,NaN,oracle.com/java/,NaN,NaN,NaN,NaN,NaN,...,James Gosling,Sun Microsystems,"May_23, 1995; 24 years ago_(1995-05-23)[1]","[Static, strong, safe, nominative, manifest]","[.class, .jar]",NaN,NaN,NaN,NaN,NaN
37,React,https://en.wikipedia.org/wiki/React_(JavaScrip...,programming_language,JavaScript library,reactjs.org,"16.12.0\n / November_14, 2019; 4 days ago_(2...",JavaScript,MIT License,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Web platform,NaN,NaN,"May_29, 2013; 6 years ago_(2013-05-29)[1]",React Repository
38,Python,https://en.wikipedia.org/wiki/Python_(programm...,programming_language,NaN,www.python.org,"[[2], [3]]",NaN,Python Software Foundation License,NaN,NaN,...,Guido van Rossum,Python Software Foundation,1990; 29_years ago_(1990)[1],"[Duck, dynamic, gradual, [4]]","[[5], [6]]",NaN,NaN,NaN,NaN,NaN


## Create relational Dataframes

In [78]:
person_list = persons_df['name'].values
person_columns = persons_df.columns.values

software_list = software_df['name'].values
software_list = [s.split(' ')[0] for s in software_list]

company_list = companies_df['name'].values
company_list = [c.split(' ')[0] for c in company_list]

#additional entities dataframes
products_df = pd.DataFrame(columns=['name', 'entity_type'])
university_df = pd.DataFrame(columns=['name', 'entity_type'])
industry_df = pd.DataFrame(columns=['name', 'entity_type'])


def fix_oculus(products):
    fixed_products = []
    for i,p in enumerate(products):
        if i == 1:
            more_products = p.split('Oculus')
            for z in more_products:
                fixed_products.append('{}{}'.format('Oculus',z))
        else:
            fixed_products.append(p)
    return fixed_products


#relations dataframes
founded_df = pd.DataFrame(columns=['name', 'company', 'founded_date'])
designed_df = pd.DataFrame(columns=['name', 'programming_language'])
written_in_df = pd.DataFrame(columns=['company','programming_language'])
key_people_df = pd.DataFrame(columns=['name', 'company', 'role'])        #TO DO 
product_of_df = pd.DataFrame(columns=['name', 'company'])
alma_mater_df = pd.DataFrame(columns=['name', 'university', 'degree'])
subsidiary_of_df = pd.DataFrame(columns=['name', 'parent_company'])
parent_of_df = pd.DataFrame(columns=['name', 'subsidiary_company'])
industry_of_df = pd.DataFrame(columns=['company', 'industry'])
completed_people = []
for index, row in companies_df.iterrows():
    company_name = row['name']
    
    #founded_by relation processing
    founded = row['founded']
    founded_by = row['founded_by']
    if type(founded) == float:
        founded = ''
    founded_date = founded.split(';')[0]
    if type(founded_by) != list:
        if type(founded_by) == float:
            founded_by = []
        else:
            founded_by =founded_by.split('  ')
    for f in founded_by:
        if f not in person_list:
            persons_df = persons_df.append({'name': f, 'entity_type': "person"}, ignore_index=True)
        founded_df = founded_df.append({'name': f, 'company': company_name, 'founded_date': founded_date}, ignore_index=True)
    
    #key_people relation processing
    key_people = row['key_people']
    if type(key_people) == float:
        key_people = []
    for i,k in enumerate(key_people):
        if '(' in k:
            person, title = k.split('(')
            title = title.replace(')', '')
            person = person.strip()
            if person != '':
                if person not in person_list:
                    persons_df = persons_df.append({'name': person, 'entity_type': 'person', 'occupation': [person, title]}, ignore_index=True)
                    key_people_df = key_people_df.append({'name': person, 'company': company_name, 'role': title}, ignore_index=True)
                    completed_people.append(person)
                else:
                    key_people_df = key_people_df.append({'name': person, 'company': company_name, 'role': title}, ignore_index=True)
                    completed_people.append(person)
            else:
                person = key_people[i-1]
                person = person.strip()
                if person not in person_list:
                    persons_df = persons_df.append({'name': person, 'entity_type': 'person', 'occupation': [person, title]}, ignore_index=True)
                    key_people_df = key_people_df.append({'name': person, 'company': company_name, 'role': title}, ignore_index=True)
                    completed_people.append(person)
                else:
                    key_people_df = key_people_df.append({'name': person, 'company': company_name, 'role': title}, ignore_index=True)
                    completed_people.append(person)
        else:
            if k not in person_list:
                if "C" in k:
                    if len(k) > 8:
                        person, title = k.rsplit('C', 1)
                        person = person.strip()
                        title = "{}{}".format('C', title)
                        if person not in person_list:
                            persons_df = persons_df.append({'name': person, 'entity_type': 'person', 'occupation': [person, title]}, ignore_index=True)
                            key_people_df = key_people_df.append({'name': person, 'company': company_name, 'role': title}, ignore_index=True)
                else:
                    person = k
                    title = key_peo
    
    #written_in relation processing
    written_in = row['written_in']
    if type(written_in) == float:
        written_in = []
    for w in written_in:
        w = re.sub(r'\[.*?]', '',w)
        if w is not '':
            if w not in software_list:
                software_df = software_df.append({'name': w, 'entity_type': 'programming_language'}, ignore_index=True)
            written_in_df = written_in_df.append({'company': company_name, 'programming_language': w}, ignore_index=True)
    
    #product_of relation processing
    products = row['products']
    if type(products) == float:
        products = []
    if company_name == 'Oculus VR':
        products = fix_oculus(products)
    for p in products:
        products_df = products_df.append({'name': p, 'entity_type': 'product'}, ignore_index=True)
        product_of_df = product_of_df.append({'name': p, 'company': company_name},ignore_index=True)
    
    #subsidiary_of and parent_of relation processing
    subsidiaries = row['subsidiaries']
    if type(subsidiaries) == float:
        subsidiaries = []
    if type(subsidiaries) == str:
        subsidiaries = [subsidiaries]
    for s in subsidiaries:
        if s not in company_list:
            companies_df = companies_df.append({'name': s, 'entity_type': 'company'}, ignore_index=True)
        subsidiary_of_df = subsidiary_of_df.append({'name': s, 'parent_company': company_name}, ignore_index=True)
        parent_of_df = parent_of_df.append({'name': company_name, 'subsidiary_company': s}, ignore_index=True)
    
    #industry_of relation processing
    industry_type = row['type']
    industries = row['industry']
    type_of_website = row['type_of_site']
    
    if type(industry_type) == float:
        industry_type = []
    if type(industry_type) == str:
        if industry_type not in ['Public', 'Division']:
            industry_type = [industry_type]
        else:
            industry_type = []
    
    if type(industries) == float:
        industries = []
    if type(industries) == str:
        industries = [industries]
    if industries == ['Conglomerate']:
        industries = []
    
    if type(type_of_website) == float:
        type_of_website = []
    
    industry_list = industry_type + industries + type_of_website
    industry_list = [i.lower() for i in industry_list]
    for i in industry_list:
        industry_df = industry_df.append({'name': i, 'entity_type': 'industry'}, ignore_index=True)
        industry_of_df = industry_of_df.append({'company': company_name, 'industry': i}, ignore_index=True)

            
for index, row in software_df.iterrows():
    #designed_by relation processing
    designed_by = row['designed_by']
    language_name = row['name']
    if type(designed_by) != list:
        if type(designed_by) == float:
            designed_by = []
        else:
            designed_by =designed_by.split('  ')
    for f in designed_by:
        if f not in person_list:
            append_df = pd.DataFrame(columns=person_columns)
            persons_df = persons_df.append({'name': f, 'entity_type': "person"}, ignore_index=True)
        designed_df = designed_df.append({'name': f, 'programming_language': language_name}, ignore_index=True)

        
for index, row in persons_df.iterrows():
    #alma_mater processing
    person_name = row['name']
    alma_mater = row['alma_mater']
    if type(alma_mater) == list:
        for i in range(len(alma_mater)):
            if "University" in alma_mater[i]:
                university = alma_mater[i]
                try:
                    if "University" not in alma_mater[i+1]:
                        degree = alma_mater[i+1]
                except:
                    degree = ''
                university_df = university_df.append({'name': university, 'entity_type': 'university'}, ignore_index=True)
                alma_mater_df = alma_mater_df.append({'name': person_name, 'university': university, 'degree': degree}, ignore_index=True)
    if type(alma_mater) == str:
        university = alma_mater
        university_df = university_df.append({'name': university, 'entity_type': 'university'}, ignore_index=True)
        degree = ''
        if '(' in university:
            university,degree = university.split('(')
            degree = degree.replace(')', '')
        alma_mater_df = alma_mater_df.append({'name': person_name, 'university': university, 'degree': degree}, ignore_index=True)
    

#clean up dfs
companies_df.drop(['founded', 'founded_by', 'written_in', 'products', 'subsidiaries', 'type', 'industry', 'type_of_site'], axis=1)
software_df.drop(['designed_by'], axis=1)
persons_df.drop(['alma_mater'], axis=1)

Michael Lynton
Director


,name,url,entity_type,website,residence,occupation,born,education,salary,net_worth,...,citizenship,other_names,employer,notable_work,died,cause_of_death,organization,awards,home_town,relatives
0,Mike Schroepfer,https://en.wikipedia.org/wiki/Mike_Schroepfer,person,NaN,San Francisco Bay area,"[CTO, Facebook]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sheryl Sandberg,https://en.wikipedia.org/wiki/Sheryl_Sandberg,person,NaN,"[Menlo Park, California, [1]]","[COO, Facebook]","Sheryl Kara Sandberg (1969-08-28) August 28, 1...","[Harvard University, BA, MBA]","[US$, [2]]",US$1.8 billion[3] (November 2019),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chris Hughes,https://en.wikipedia.org/wiki/Chris_Hughes,person,NaN,NaN,Entrepreneur,"Christopher Hughes (1983-11-26) November 26, 1...",Phillips Academy,NaN,"[US$, [1]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Evan Spiegel,https://en.wikipedia.org/wiki/Evan_Spiegel,person,NaN,"[Brentwood, Los Angeles, California, [1]]",Entrepreneur,"[Los Angeles, California]",Crossroads School for Arts and Sciences,NaN,"[US$, [2]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Evan Williams,https://en.wikipedia.org/wiki/Evan_Williams_(I...,person,NaN,"San Francisco, California, US",Internet Entrepreneur,"[Clarks, Nebraska]",NaN,NaN,"[billion, [1]]",...,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Omid Kordestani,https://en.wikipedia.org/wiki/Omid_Kordestani,person,NaN,"[Atherton, California, [3]]","[Executive Chairman, Twitter]","[Tehran, [1], Iran, [2]]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Biz Stone,https://en.wikipedia.org/wiki/Biz_Stone,person,NaN,"Marin County, California","[Creative director, software engineer, busines...",NaN,NaN,NaN,NaN,...,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Jack Dorsey,https://en.wikipedia.org/wiki/Jack_Dorsey,person,NaN,"[San Francisco, California]","[Programmer, internet entrepreneur]","[[1], St. Louis, Missouri]",NaN,"[Square, Inc., Twitter, [2]]","[US$, [3]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Steve Huffman,https://en.wikipedia.org/wiki/Steve_Huffman,person,NaN,"San Francisco, California","[Co-founder and CEO of Reddit, Co-founder of H...",NaN,Computer Science,NaN,NaN,...,NaN,spez[1],Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Alexis Ohanian,https://en.wikipedia.org/wiki/Alexis_Ohanian,person,Alexisohanian.com,"[West Palm Beach, Florida, citation needed]","[Entrepreneur, venture capitalist, investor]","Alexis Kerry Ohanian (1983-04-24) April 24, 19...",NaN,NaN,NaN,...,NaN,NaN,NaN,"[Reddit, Initialized Capital]",NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
founded_df

,name,company,founded_date
0,Mark Zuckerberg,Facebook,"February_4, 2004"
1,Eduardo Saverin,Facebook,"February_4, 2004"
2,Andrew McCollum,Facebook,"February_4, 2004"
3,Dustin Moskovitz,Facebook,"February_4, 2004"
4,Chris Hughes,Facebook,"February_4, 2004"
5,Mircea Paşoi,Summify,
6,Cristian Strat,Summify,
7,Kayvon Beykpour,Periscope,
8,Joe Bernstein,Periscope,
9,Aaron Wasserman,Periscope,


In [10]:
designed_df

,name,programming_language
0,Brendan Eich,JavaScript
1,Martin Odersky,Scala
2,Yukihiro Matsumoto,Ruby
3,James Gosling,Java
4,Guido van Rossum,Python


In [11]:
written_in_df

,company,programming_language
0,Periscope,Go
1,Periscope,server-side
2,Reddit,Python
3,Reddit,React
4,Twitter,Java
5,Twitter,Ruby
6,Twitter,Scala
7,Twitter,JavaScript
8,WhatsApp,Erlang


In [21]:
product_of_df

,name,company
0,Messenger,Facebook
1,Watch,Facebook
2,Portal,Facebook
3,Samsung Gear VR,Oculus VR
4,Oculus,Oculus VR
5,Oculus Rift CV1,Oculus VR
6,Oculus Go,Oculus VR
7,Oculus Rift S,Oculus VR
8,Oculus Quest,Oculus VR
9,Oculus Medium,Oculus VR


In [26]:
alma_mater_df

,name,university,degree
0,Mike Schroepfer,Stanford University,
1,Chris Hughes,Harvard University,
2,Evan Spiegel,Stanford University,
3,Omid Kordestani,San Jose State University,BS
4,Omid Kordestani,Stanford University,MBA
5,Jack Dorsey,University of Missouri–Rolla,MBA
6,Jack Dorsey,New York University,
7,Steve Huffman,University of Virginia,
8,Alexis Ohanian,University of Virginia,
9,Aaron Swartz,Stanford University,


In [24]:
dedup_df(fix_name(university_df))

,name,entity_type
0,Stanford University,university
1,Harvard University,university
3,San Jose State University,university
5,University of Missouri–Rolla,university
6,New York University,university
7,University of Virginia,university


In [394]:
parent_of_df

,name,subsidiary_company
0,Facebook,Instagram
1,Facebook,WhatsApp
2,Facebook,Oculus
3,Facebook,Calibra
4,Twitter,Vine
5,Twitter,Periscope
6,Twitter,MoPub
7,Oculus VR,Oculus Studios


In [395]:
subsidiary_of_df

,name,parent_company
0,Instagram,Facebook
1,WhatsApp,Facebook
2,Oculus,Facebook
3,Calibra,Facebook
4,Vine,Twitter
5,Periscope,Twitter
6,MoPub,Twitter
7,Oculus Studios,Oculus VR


In [25]:
industry_of_df

,company,industry
0,Facebook,social media
1,Facebook,advertising
2,Summify,news aggregation
3,Summify,social media
4,Periscope,video streaming
5,Periscope,social networking service
6,Vine,video sharing
7,Snapchat,photo sharing
8,Snapchat,instant messaging
9,Snapchat,video chat


In [79]:
key_people_df

,name,company,role
0,Mark Zuckerberg,Facebook,Chairman and CEO
1,Sheryl Sandberg,Facebook,COO
2,David Wehner,Facebook,CFO
3,Mike Schroepfer,Facebook,CTO
4,Steve Huffman,Reddit,co-founder and CEO
5,Omid Kordestani,Twitter,Executive Chairman
6,Jack Dorsey,Twitter,CEO
7,Ned Segal,Twitter,CFO
8,Andrew Bosworth,Oculus VR,"Vice President, VR/AR"
9,Ma Huateng,Tencent,"Chairman, CEO"
